In [1]:

use_colab = True
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive._mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/685_Project'

    !pip install -r "/content/drive/My Drive/685_Project/requirements.txt"

!pip install sentencepiece

from datasets import load_dataset


import sys
import os
import random
import shutil
import copy
import inspect

os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

import numpy as np
import torch
import transformers
import datasets
import sklearn.metrics
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tqdm
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer



Mounted at /content/drive
/content/drive/My Drive/685_Project
     |████████████████████████████████| 298 kB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
     |████████████████████████████████| 61 kB 482 kB/s 
     |████████████████████████████████| 243 kB 35.0 MB/s 
     |████████████████████████████████| 132 kB 33.7 MB/s 
     |████████████████████████████████| 1.1 MB 36.5 MB/s 
     |████████████████████████████████| 596 kB 32.9 MB/s 
     |████████████████████████████████| 895 kB 36.6 MB/s 
     |████████████████████████████████| 3.3 MB 39.3 MB/s 
     |████████████████████████████████| 160 kB 49.6 MB/s 
     |████████████████████████████████| 192 kB 47.3 MB/s 
     |████████████████████████████████| 271 kB 46.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [2]:
assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla K80, n_gpu: 1


In [3]:
def mappingFunction(index, Data):
# NOT         0 0 0 -> 0
# OFF UNT     1 1 0 -> 1
# OFF TIN IND 1 2 1 -> 2
# OFF TIN GRP 1 2 2 -> 3
# OFF TIN OTH 1 2 3 -> 4
  if Data.iloc[index][2] == "NOT":
    return 0
  elif Data.iloc[index][2] == "OFF" and Data.iloc[index][3] == "UNT" :
    return 1
  elif Data.iloc[index][2] == "OFF" and Data.iloc[index][3] == "TIN" and Data.iloc[index][4] == "IND":
    return 2
  elif Data.iloc[index][2] == "OFF" and Data.iloc[index][3] == "TIN" and Data.iloc[index][4] == "GRP":
    return 3
  elif Data.iloc[index][2] == "OFF" and Data.iloc[index][3] == "TIN" and Data.iloc[index][4] == "OTH":
    return 4

In [4]:
import pandas as pd
train = pd.read_csv('/content/drive/My Drive/685_Project/olid_train_v2.csv',sep='\t')

# Convert to Multi-Class
train["class"] = -1
for index in range(len(train)):
  train.at[index,"class"] =  mappingFunction(index, train)


val = train.sample(frac=0.065,replace=False)
final = pd.concat([train, val, val]).drop_duplicates(keep=False)


print(train.head())
#sanity checks
assert len(final) == len(train) - len(val)
s1 = pd.merge(val, final, how='inner', on=['id'])
assert len(s1)==0

s1 = pd.merge(final, train, how='inner', on=['id'])
assert len(s1) == len(final)

s1 = pd.merge(val, train, how='inner', on=['id'])
assert len(s1) == len(val)

val.to_csv('olid_val_v2.csv',sep='\t')
final.to_csv('olid_train_v3.csv',sep='\t')

train_dataset = load_dataset('csv', data_files={'train': './olid_train_v3.csv','valid': 'olid_val_v2.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_a_dataset = load_dataset('csv', data_files={'test': '/content/drive/My Drive/685_Project/olid_test_subtask_a.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_b_dataset = load_dataset('csv', data_files={'test': '/content/drive/My Drive/685_Project/olid_test_subtask_b.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_c_dataset = load_dataset('csv', data_files={'test': '/content/drive/My Drive/685_Project/olid_test_subtask_c.csv'}, delimiter="\t", cache_dir='./olid_cache')


      id                                              tweet  ...    subtask_c class
0  86426  @USER She should ask a few native Americans wh...  ...  not present     1
1  90194  @USER @USER Go home you’re drunk!!! @USER #MAG...  ...          IND     2
2  16820  Amazon is investigating Chinese employees who ...  ...  not present     0
3  62688  @USER Someone should'veTaken" this piece of sh...  ...  not present     1
4  43605  @USER @USER Obama wanted liberals &amp; illega...  ...  not present     0

[5 rows x 6 columns]


Using custom data configuration default-8defac0403905212


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to ./olid_cache/csv/default-8defac0403905212/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


...

  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-69e8990fed7492fc
Reusing dataset csv (./olid_cache/csv/default-69e8990fed7492fc/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-609ab11a210e5d0a
Reusing dataset csv (./olid_cache/csv/default-609ab11a210e5d0a/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
set_seed(685)

train_dataloader = torch.utils.data.DataLoader(train_dataset['train'], shuffle=True, batch_size=8)
val_dataloader = torch.utils.data.DataLoader(train_dataset['valid'], shuffle=True, batch_size=8)
test_dataloader = torch.utils.data.DataLoader(test_a_dataset['test'], shuffle=True)

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', 
    num_labels = 5, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    cache_dir='./roberta_cache'
)
model.to(device)

batch_size = 100
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
epochs = 5

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [9]:
# os.mkdir("trained_model_offenseval_baseline_multiclass")

In [10]:
import numpy as np
y_pred =[]
y_true = []
incorrect = []
# function to get validation accuracy
def get_validation_performance(val_dataloader):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_correct = 0

    for batch in val_dataloader:
      
      if len(batch) == 0: continue
      inputs = tokenizer(batch['tweet'],padding='longest',return_tensors="pt").to(device)
      labels = batch['class'].to(device)
      
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        outputs = model(**inputs)
        loss = outputs.loss
        logits = outputs.logits
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct
        incorrect.extend([batch["tweet"][index] for index in range(len(batch["tweet"])) if pred_flat[index] != labels_flat[index]])
        y_pred.extend(pred_flat)
        y_true.extend(labels_flat)
        
        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / (len(val_dataloader) * val_dataloader.batch_size)
    return avg_val_accuracy, incorrect, y_pred, y_true

In [11]:
import random

# training loop

# For each epoch...
for epoch_i in range(0, epochs):
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...

    for batch in tqdm.notebook.tqdm(train_dataloader):
      if len(batch) == 0: continue

      inputs = tokenizer(batch['tweet'],padding='longest',return_tensors="pt").to(device)
      labels = batch['class'].to(device)
      # Clear the previously calculated gradient
      model.zero_grad()        

      # Perform a forward pass (evaluate the model on this training batch).
      outputs = model(**inputs, labels=labels)
      loss = outputs.loss
      logits = outputs.logits

      total_train_loss += loss.item()

      # Perform a backward pass to calculate the gradients.
      loss.backward()

      # Update parameters and take a step using the computed gradient.
      optimizer.step()
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set. Implement this function in the cell above.
    print(f"Total loss: {total_train_loss}")
    val_acc, _, _, _ = get_validation_performance(val_dataloader)
    print(f"Validation accuracy: {val_acc}")
    torch.save(model.state_dict(),os.path.join('./trained_model_offenseval_baseline_multiclass/', 'multiclass_baseline_{}.pt'.format(epoch_i)))
    
print("")
print("Training complete!")



======== Epoch 1 / 5 ========
Training...


  0%|          | 0/1548 [00:00<?, ?it/s]

Total loss: 1359.6751976460218
Validation accuracy: 0.6631944444444444

======== Epoch 2 / 5 ========
Training...


  0%|          | 0/1548 [00:00<?, ?it/s]

Total loss: 1585.0797228515148
Validation accuracy: 0.6412037037037037

======== Epoch 3 / 5 ========
Training...


...

Total loss: 1580.9335772395134
Validation accuracy: 0.6412037037037037

======== Epoch 4 / 5 ========
Training...


  0%|          | 0/1548 [00:00<?, ?it/s]

Total loss: 1578.0017941594124
Validation accuracy: 0.6412037037037037

======== Epoch 5 / 5 ========
Training...


  0%|          | 0/1548 [00:00<?, ?it/s]

Total loss: 1579.6113730669022
Validation accuracy: 0.6412037037037037

Training complete!


In [12]:
import pandas as pd

test_a = pd.read_csv('/content/drive/My Drive/685_Project/olid_test_subtask_a.csv',sep='\t')
test_b = pd.read_csv('/content/drive/My Drive/685_Project/olid_test_subtask_b.csv',sep='\t')
test_c = pd.read_csv('/content/drive/My Drive/685_Project/olid_test_subtask_c.csv',sep='\t')


test_a = test_a.rename(columns = {"label": "label_a"})
test_b = test_b.rename(columns = {"label": "label_b"})
test_c = test_c.rename(columns = {"label": "label_c"})


test = test_a.merge(test_b, on="id", how = 'outer')
test = test.merge(test_c , on ="id", how = 'outer')

# drop extra columns
test = test.drop(columns=['tweet', 'tweet_y'])

# rename tweet_x  with tweet
test = test.rename(columns = {"tweet_x": "tweet"})



In [13]:
#################
test["class"] = -1
for index in range(len(test)):
  test.at[index,"class"] =  mappingFunction(index, test)
#################
# print(test.head())

In [14]:
test = test.replace(np.nan, 'not present', regex=True)

In [15]:
test

,id,tweet,label_a,label_b,label_c,class
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,TIN,OTH,4
1,27014,"#ConstitutionDay is revered by Conservatives, ...",NOT,not present,not present,0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT,not present,not present,0
3,13876,#Watching #Boomer getting the news that she is...,NOT,not present,not present,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,OFF,TIN,GRP,3
...,...,...,...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,OFF,TIN,GRP,3
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,NOT,not present,not present,0
857,67018,3 people just unfollowed me for talking about ...,OFF,UNT,not present,1
858,50665,#WednesdayWisdom Antifa calls the right fascis...,NOT,not present,not present,0


In [16]:
test.to_csv('olid_test_multiclass_v2.csv',sep='\t')
test_a_dataset = load_dataset('csv', data_files={'test': './olid_test_multiclass_v2.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_dataloader = torch.utils.data.DataLoader(test_a_dataset['test'], shuffle=True)

Using custom data configuration default-20c008c1486f63f5


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to ./olid_cache/csv/default-20c008c1486f63f5/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
acc,incorrect_examples, y_pred, y_true =  get_validation_performance(test_dataloader) 

In [18]:
from sklearn.metrics import classification_report
report = classification_report(y_true=y_true, y_pred=y_pred)
print(report)


              precision    recall  f1-score   support

           0       0.68      0.97      0.80      3390
           1       0.00      0.00      0.00       252
           2       0.39      0.12      0.19       915
           3       0.41      0.02      0.03       438
           4       0.00      0.00      0.00       170

    accuracy                           0.66      5165
   macro avg       0.30      0.22      0.20      5165
weighted avg       0.55      0.66      0.56      5165



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
np.savetxt("incorrect_examples_offenseval_multiclass.csv", 
           incorrect,
           delimiter =", ", 
           fmt ='% s')

In [21]:
from google.colab import files
files.download("incorrect_examples_offenseval_multiclass.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>